In [ ]:
# prompt: create afolder by the name of audio in colab content dir

!mkdir /content/audio_1

In [ ]:
# prompt: code to unzip /content/tiktok.zip

!unzip /content/TikTok_videos.zip -d /content/my_folder_1

unzip:  cannot find or open /content/TikTok_videos.zip, /content/TikTok_videos.zip.zip or /content/TikTok_videos.zip.ZIP.


In [ ]:
# @title Extract Audio from Videos
# @markdown This code extracts audio from all MP4 files in a specified folder,
# @markdown creates an "audio_folder", and saves the audio files with "_audio"
# @markdown appended to the original name.

# Install the necessary library
!pip install moviepy

import os
from moviepy.editor import *

# --- Configuration ---
# @markdown **Enter the name of the folder containing your MP4 videos:**
video_folder = '/content/my_folder_1/TikTok_videos'  # @param {type:"string"}

# @markdown **Enter the name for the folder to save the audio files:**
audio_output_folder = '/content/audio_1'  # @param {type:"string"}

# --- Code Execution ---

# Create the output folder if it doesn't exist
if not os.path.exists(audio_output_folder):
    os.makedirs(audio_output_folder)
    print(f"Created directory: {audio_output_folder}")

# Get a list of all files in the video folder
try:
    video_files = os.listdir(video_folder)
except FileNotFoundError:
    print(f"Error: The folder '{video_folder}' was not found. Please make sure you have uploaded it and entered the correct name.")
    video_files = []


# Process each video file
for video_file in video_files:
    if video_file.endswith(".mp4"):
        video_path = os.path.join(video_folder, video_file)

        # Get the filename without the extension
        file_name_without_ext = os.path.splitext(video_file)[0]

        # Define the output audio file name
        audio_file_name = f"{file_name_without_ext}_audio.mp3"
        audio_path = os.path.join(audio_output_folder, audio_file_name)

        print(f"Extracting audio from: {video_file}")

        try:
            # Load the video file. [3]
            video_clip = VideoFileClip(video_path)

            # Extract the audio and write it to the new file. [3]
            video_clip.audio.write_audiofile(audio_path)

            video_clip.close()
            print(f"Successfully saved audio to: {audio_path}")

        except Exception as e:
            print(f"An error occurred while processing {video_file}: {e}")

print("\nAudio extraction process complete.")

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



Error: The folder '/content/my_folder_1/TikTok_videos' was not found. Please make sure you have uploaded it and entered the correct name.

Audio extraction process complete.


In [ ]:
# prompt: code to unzip a folder /content/audio to audio.zip

!unzip "/content/audio (1).zip" -d /content/audio_extracted

Archive:  /content/audio (1).zip
replace /content/audio_extracted/content/audio_1/Download (29)_audio.mp3? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# # @title Transcribe Audio with Hugging Face Transformers
# # @markdown This code uses the Hugging Face pipeline for transcription.

# # Install necessary libraries
# !pip install transformers accelerate

# from transformers import pipeline
# import torch

# # @markdown **Enter the path to your audio file:**
# audio_file_path = 'audio_folder/tiktok_dave_mik_6951524182735867142_audio.mp3'  # @param {type:"string"}

# # Check if a GPU is available and set the device
# device = "cuda:0" if torch.cuda.is_available() else "cpu"

# # Initialize the ASR pipeline
# # Using Whisper for this example
# print("Initializing ASR pipeline...")
# pipe = pipeline(
#   "automatic-speech-recognition",
#   model="openai/whisper-base", # You can change this to "openai/whisper-large-v3" for higher accuracy
#   device=device
# )

# print(f"Transcribing {audio_file_path}...")
# # Transcribe the audio file
# # The pipeline automatically handles loading and resampling the audio
# transcription = pipe(audio_file_path)

# print("\n--- Transcription Result ---")
# print(transcription["text"])

In [ ]:
# # @title Transcribe Audio with OpenAI's Whisper
# # @markdown This code installs the Whisper library and transcribes an audio file.

# # Install the library
# !pip install -U openai-whisper

# import whisper

# # @markdown **Enter the path to your audio file:**
# audio_file_path = 'audio_folder/tiktok_dave_mik_6951524182735867142_audio.mp3'  # @param {type:"string"}

# # @markdown **Select the Whisper model size:**
# model_size = 'base'  # @param ["tiny", "base", "small", "medium", "large"]

# print("Loading Whisper model...")
# # Load the model (the first time you run this, it will download the model)
# model = whisper.load_model(model_size)

# print(f"Transcribing {audio_file_path}...")
# # Transcribe the audio file
# result = model.transcribe(audio_file_path)

# print("\n--- Transcription Result ---")
# print(result["text"])

In [ ]:
# @title ASR Model Comparison: Native Whisper vs. Hugging Face Whisper
# @markdown This script transcribes audio files using two methods for the same model:
# @markdown 1. The original `openai-whisper` library.
# @markdown 2. The Hugging Face `transformers` pipeline.
# @markdown It then calculates perplexity scores for each transcription and saves the results.

# --- 1. Installation of All Necessary Libraries ---
print("--- Installing Libraries ---")
# Using -q for quieter installation
!pip install -q -U openai-whisper
!pip install -q transformers accelerate torch

import os
import torch
import whisper
from transformers import pipeline, BertTokenizer, BertForMaskedLM
import numpy as np
import re
import gc # For garbage collection to manage memory

# --- 2. Configuration ---
# @markdown ### **Step 1: Set your paths and model size**
# @markdown ---
# @markdown **Enter the path to your folder of audio files in Colab:**
audio_folder_path = '/content/audio_1'  # @param {type:"string"}
# @markdown **Select the Whisper model size for a fair comparison:**
whisper_model_size = 'base'  # @param ["tiny", "base", "small", "medium", "large"]


# --- 3. Helper Function for Perplexity Calculation ---
# This function is defined once and used for both models' outputs.
def calculate_perplexity(text, model, tokenizer, device):
    """Calculates the perplexity of a text using a BERT model."""
    if not text or not isinstance(text, str) or not text.strip():
        return float('nan') # Return NaN for empty or invalid strings

    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize the text
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)

    if input_ids.size(1) == 0:
        return float('nan')

    # Calculate loss using the model
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss

    # Perplexity is the exponential of the loss
    perplexity = torch.exp(loss)
    return perplexity.item()


# --- 4. Main Processing ---
def run_comparison():
    print("\n--- Initializing Models (this may take a while) ---")

    # Check for GPU availability and set device
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    if device == "cuda:0":
        print("✅ GPU detected! Using CUDA for faster processing.")
    else:
        print("⚠️ No GPU detected. Using CPU, which will be significantly slower.")

    # Initialize BERT for Perplexity Calculation (once)
    print("Loading BERT model for perplexity: bert-base-uncased...")
    perplexity_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    perplexity_model = BertForMaskedLM.from_pretrained('bert-base-uncased').to(device)

    # File handlers
    output_file_native = 'native_whisper_results.txt'
    output_file_hf = 'huggingface_whisper_results.txt'

    # Get list of audio files
    try:
        audio_files = [f for f in os.listdir(audio_folder_path) if f.endswith(('.mp3', '.wav', '.m4a', '.mp4'))]
        if not audio_files:
            print(f"\n❌ Error: No audio files (.mp3, .wav, .m4a, .mp4) found in '{audio_folder_path}'")
            return
    except FileNotFoundError:
        print(f"\n❌ Error: The folder '{audio_folder_path}' was not found. Please check the path and re-run.")
        return

    with open(output_file_native, 'w') as f_native, open(output_file_hf, 'w') as f_hf:
        # Write headers to both files
        header = f"{'Filename':<50} | {'Avg. Confidence':<20} | {'Perplexity (BERT)':<20} | {'Transcription'}\n"
        f_native.write("--- Results from Native 'openai-whisper' Library ---\n" + header + "="*120 + "\n")
        f_hf.write("--- Results from Hugging Face 'transformers' Pipeline ---\n" + header + "="*120 + "\n")

        # --- MODEL 1: NATIVE WHISPER LIBRARY ---
        print(f"\n--- Loading Native Whisper Model: {whisper_model_size} ---")
        native_whisper_model = whisper.load_model(whisper_model_size, device=device)

        for filename in audio_files:
            print(f"Processing '{filename}' with Native Whisper...")
            audio_path = os.path.join(audio_folder_path, filename)
            try:
                result = native_whisper_model.transcribe(audio_path, fp16=torch.cuda.is_available())
                text = result['text']

                # Calculate average confidence from segment probabilities
                confidences = [segment.get('avg_logprob') for segment in result['segments'] if 'avg_logprob' in segment]
                avg_confidence = np.exp(np.mean(confidences)) if confidences else float('nan')

                perplexity = calculate_perplexity(text, perplexity_model, perplexity_tokenizer, device)
                f_native.write(f"{filename:<50} | {avg_confidence:<20.4f} | {perplexity:<20.2f} | {text}\n")

            except Exception as e:
                f_native.write(f"{filename:<50} | ERROR | ERROR | {str(e)}\n")

        # Clear memory before loading the next model
        del native_whisper_model
        gc.collect()
        torch.cuda.empty_cache()
        print("\n--- Native Whisper processing complete. ---")


        # --- MODEL 2: HUGGING FACE TRANSFORMERS PIPELINE ---
        hf_model_name = f"openai/whisper-{whisper_model_size}"
        print(f"\n--- Loading Hugging Face Whisper Model: {hf_model_name} ---")
        hf_pipeline = pipeline(
            "automatic-speech-recognition",
            model=hf_model_name,
            device=device
        )

        for filename in audio_files:
            print(f"Processing '{filename}' with Hugging Face Pipeline...")
            audio_path = os.path.join(audio_folder_path, filename)
            try:
                # The basic HF pipeline does not easily return segment-level confidence.
                # This is a key practical difference between the libraries.
                result = hf_pipeline(audio_path)
                text = result['text']
                confidence_hf = "N/A" # Marking as not available for a fair comparison of ease-of-use.

                perplexity = calculate_perplexity(text, perplexity_model, perplexity_tokenizer, device)
                f_hf.write(f"{filename:<50} | {confidence_hf:<20} | {perplexity:<20.2f} | {text}\n")

            except Exception as e:
                f_hf.write(f"{filename:<50} | ERROR | ERROR | {str(e)}\n")

    print(f"\n✅ --- Processing Complete! ---")
    print(f"Results saved to '{output_file_native}' and '{output_file_hf}'.")


# --- Run the entire comparison process ---
run_comparison()

--- Installing Libraries ---
^C


<!-- Better comaprison -->

Better and consistant Comparison

In [ ]:
# @title Cell #1: Native Whisper Transcription
# --- 1. Installation & Setup ---
print("--- Installing Libraries ---")
!pip install -q -U openai-whisper
!pip install -q transformers accelerate torch sentencepiece
!pip install -q --no-deps tabulate
print("\n--- Ensuring ffmpeg is installed ---")
!sudo apt-get update > /dev/null 2>&1 && sudo apt-get install -y ffmpeg > /dev/null 2>&1

import os
import torch
import whisper
import numpy as np
import re
import gc
from tabulate import tabulate
from transformers import BertTokenizer, BertForMaskedLM

# --- 2. Configuration ---
audio_folder_path = '/content/audio_extracted/content/audio_1'
whisper_model_size = 'base'

# --- 3. Helper Function for Perplexity ---
def calculate_perplexity(text, model, tokenizer, device):
    if not text or not isinstance(text, str) or not text.strip():
        return float('nan')
    text = re.sub(r'\s+', ' ', text).strip()
    input_ids = tokenizer.encode(text, return_tensors='pt', truncation=True, max_length=512).to(device)
    if input_ids.size(1) == 0:
        return float('nan')
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
    if loss is not None:
        return torch.exp(loss).item()
    return float('nan')

# --- 4. Main Processing Logic ---
def run_native_whisper():
    print("\n--- Initializing Models ---")
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"✅ Using device: {device}")

    print("Loading BERT model for perplexity...")
    perplexity_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    perplexity_model = BertForMaskedLM.from_pretrained('bert-base-uncased').to(device)
    perplexity_model.eval()

    try:
        audio_files = sorted([f for f in os.listdir(audio_folder_path) if f.endswith(('.mp3', '.wav', '.m4a', '.mp4'))])
        if not audio_files:
            print(f"❌ Error: No audio files found in '{audio_folder_path}'")
            return
    except FileNotFoundError:
        print(f"❌ Error: The folder '{audio_folder_path}' was not found.")
        return

    results_data = []

    print(f"\n--- Loading Native Whisper Model: {whisper_model_size} ---")
    native_whisper_model = whisper.load_model(whisper_model_size, device=device)

    for filename in audio_files:
        print(f"Processing '{filename}'...")
        audio_path = os.path.join(audio_folder_path, filename)

        transcription_text = "Error"
        perplexity_score = float('nan')

        try:
            result = native_whisper_model.transcribe(audio_path, verbose=False)
            transcription_text = result.get('text', '').strip() or "No speech detected."
            perplexity_score = calculate_perplexity(transcription_text, perplexity_model, perplexity_tokenizer, device)
        except Exception as e:
            transcription_text = f"ERROR: {str(e)}"

        results_data.append([
            filename,
            'Native Whisper',
            'nan', # Confidence score is not available
            f"{perplexity_score:.2f}" if not np.isnan(perplexity_score) else 'nan',
            transcription_text
        ])

    # --- 5. Displaying Results ---
    print("\n✅ --- Native Whisper Processing Complete! ---")
    headers = ["Filename", "Model", "Avg. Confidence", "Perplexity", "Transcription"]
    print(tabulate(results_data, headers=headers, tablefmt="grid", maxcolwidths=[40, 15, 15, 15, 80]))

    # Return the data for the final unified table
    return results_data

# --- Execute the script and store the results ---
native_results = run_native_whisper()

--- Installing Libraries ---

--- Ensuring ffmpeg is installed ---

--- Initializing Models ---
✅ Using device: cuda:0
Loading BERT model for perplexity...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



--- Loading Native Whisper Model: base ---
Processing 'Download (1)_audio.mp3'...
Detected language: English


100%|██████████| 6715/6715 [00:16<00:00, 414.00frames/s]


Processing 'Download (10)_audio.mp3'...
Detected language: English


...

Processing 'tiktok_trippintrevor_7009029789965110533_audio.mp3'...
Detected language: English


100%|██████████| 1050/1050 [00:00<00:00, 2715.43frames/s]


Processing 'tiktok_x1midnightlovex_7010121683033918725_audio.mp3'...
Detected language: English


100%|██████████| 880/880 [00:00<00:00, 1813.72frames/s]


✅ --- Native Whisper Processing Complete! ---
+------------------------------------------+----------------+-------------------+----------------+----------------------------------------------------------------------------------+
| Filename                                 | Model          |   Avg. Confidence |     Perplexity | Transcription                                                                    |
+==========================================+================+===================+================+==================================================================================+
| Download (1)_audio.mp3                   | Native Whisper |               nan |    1.41        | It's an hour in and it's um... I'm pretty uncomfortable. You gotta be honest.    |
|                                          |                |                   |                | Um, the first hour it was very difficult to try and like what position do I want |
|                                          

In [ ]:
# @title Cell #2: Hugging Face Whisper Transcription
# --- 1. Installation & Setup ---
print("--- Installing Libraries ---")
!pip install -q -U transformers accelerate
!pip install -q librosa sentencepiece
!pip install -q --no-deps tabulate
print("\n--- Libraries installed ---")

import os
import torch
import numpy as np
import re
import gc
import librosa
from tabulate import tabulate
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
from transformers import BertTokenizer, BertForMaskedLM

# --- 2. Configuration ---
audio_folder_path = '/content/audio_extracted/content/audio_1'
whisper_model_size = 'base'

# --- 3. Helper Function for Perplexity ---
def calculate_perplexity(text, model, tokenizer, device):
    if not text or not isinstance(text, str) or not text.strip():
        return float('nan')
    text = re.sub(r'\s+', ' ', text).strip()
    input_ids = tokenizer.encode(text, return_tensors='pt', truncation=True, max_length=512).to(device)
    if input_ids.size(1) == 0:
        return float('nan')
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
    if loss is not None:
        return torch.exp(loss).item()
    return float('nan')

# --- 4. Main Processing Logic ---
def run_huggingface_whisper():
    print("\n--- Initializing Models ---")
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"✅ Using device: {device}")

    print("Loading BERT model for perplexity...")
    perplexity_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    perplexity_model = BertForMaskedLM.from_pretrained('bert-base-uncased').to(device)
    perplexity_model.eval()

    try:
        audio_files = sorted([f for f in os.listdir(audio_folder_path) if f.endswith(('.mp3', '.wav', '.m4a', '.mp4'))])
        if not audio_files:
            print(f"❌ Error: No audio files found in '{audio_folder_path}'")
            return
    except FileNotFoundError:
        print(f"❌ Error: The folder '{audio_folder_path}' was not found.")
        return

    results_data = []

    print(f"\n--- Loading Hugging Face Whisper Model: openai/whisper-{whisper_model_size} ---")
    hf_model_id = f"openai/whisper-{whisper_model_size}"
    hf_model = AutoModelForSpeechSeq2Seq.from_pretrained(hf_model_id).to(device)
    hf_processor = AutoProcessor.from_pretrained(hf_model_id)

    for filename in audio_files:
        print(f"Processing '{filename}'...")
        audio_path = os.path.join(audio_folder_path, filename)

        transcription_text = "Error"
        perplexity_score = float('nan')
        confidence = float('nan')

        try:
            audio_input, sample_rate = librosa.load(audio_path, sr=16000)
            input_features = hf_processor(audio_input, sampling_rate=16000, return_tensors="pt").input_features.to(device)

            with torch.no_grad():
                output = hf_model.generate(
                    input_features,
                    return_dict_in_generate=True,
                    output_scores=True
                )

            transcription_text = hf_processor.batch_decode(output.sequences, skip_special_tokens=True)[0].strip() or "No speech detected."

            # --- CONFIDENCE CALCULATION WITH FIX ---
            if hasattr(output, 'scores') and output.scores is not None:
                scores = torch.stack(output.scores, dim=1)
                log_softmax_scores = torch.nn.functional.log_softmax(scores, dim=-1)

                generated_tokens = output.sequences[:, 1:]

                # **FIX:** Truncate generated_tokens to match the length of the scores.
                # This resolves the "Size does not match at dimension 1" error.
                num_scores = scores.shape[1]
                generated_tokens = generated_tokens[:, :num_scores]

                token_log_probs = torch.gather(log_softmax_scores, 2, generated_tokens.unsqueeze(-1)).squeeze(-1)
                log_probs = token_log_probs.cpu().numpy().flatten().tolist()

                if log_probs:
                    avg_log_prob = np.mean(log_probs)
                    confidence = np.exp(avg_log_prob)

            perplexity_score = calculate_perplexity(transcription_text, perplexity_model, perplexity_tokenizer, device)

        except Exception as e:
            transcription_text = f"ERROR: {str(e)}"

        results_data.append([
            filename,
            'Hugging Face',
            f"{confidence:.4f}" if not np.isnan(confidence) else 'nan',
            f"{perplexity_score:.2f}" if not np.isnan(perplexity_score) else 'nan',
            transcription_text
        ])

    # --- 5. Displaying Results ---
    print("\n✅ --- Hugging Face Processing Complete! ---")
    headers = ["Filename", "Model", "Avg. Confidence", "Perplexity", "Transcription"]
    print(tabulate(results_data, headers=headers, tablefmt="grid", maxcolwidths=[40, 15, 15, 15, 80]))

    # Return the data for the final unified table
    return results_data

# --- Execute the script and store the results ---
hf_results = run_huggingface_whisper()

--- Installing Libraries ---

--- Libraries installed ---

--- Initializing Models ---
✅ Using device: cuda:0
Loading BERT model for perplexity...


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



--- Loading Hugging Face Whisper Model: openai/whisper-base ---


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Processing 'Download (1)_audio.mp3'...
Processing 'Download (10)_audio.mp3'...
Processing 'Download (11)_audio.mp3'...
Processing 'Download (12)_audio.mp3'...
Processing 'Download (13)_audio.mp3'...
Processing 'Download (14)_audio.mp3'...
Processing 'Download (15)_audio.mp3'...
Processing 'Download (16)_audio.mp3'...
Processing 'Download (17)_audio.mp3'...
Processing 'Download (18)_audio.mp3'...
Processing 'Download (19)_audio.mp3'...
Processing 'Download (2)_audio.mp3'...
Processing 'Download (20)_audio.mp3'...
Processing 'Download (21)_audio.mp3'...
Processing 'Download (22)_audio.mp3'...
Processing 'Download (23)_audio.mp3'...
Processing 'Download (24)_audio.mp3'...
Processing 'Download (25)_audio.mp3'...
Processing 'Download (26)_audio.mp3'...
Processing 'Download (27)_audio.mp3'...
Processing 'Download (28)_audio.mp3'...
Processing 'Download (29)_audio.mp3'...
Processing 'Download (3)_audio.mp3'...
Processing 'Download (30)_audio.mp3'...
Processing 'Download (4)_audio.mp3'...
Proc

In [ ]:
# @title Cell #3: Unified Comparison Table
from tabulate import tabulate
import re

# This cell assumes 'native_results' and 'hf_results' exist from running the cells above.
if 'native_results' in locals() and 'hf_results' in locals():
    print("--- Combining results from both models ---")

    # Combine the two lists of results
    all_results = native_results + hf_results

    # Group results by filename for side-by-side comparison
    grouped_results = {}
    for row in all_results:
        # row is a list: [filename, model, confidence, perplexity, transcription]
        filename = row[0]
        if filename not in grouped_results:
            grouped_results[filename] = []
        grouped_results[filename].append(row)

    # Prepare data for the final tabulation
    final_table_data = []
    headers = ["Filename", "Model", "Avg. Confidence", "Perplexity", "Transcription"]

    # Sort filenames to ensure a consistent order
    try:
        sorted_filenames = sorted(list(grouped_results.keys()), key=lambda x: int(re.search(r'\((\d+)\)', x).group(1)) if re.search(r'\((\d+)\)', x) else float('inf'))
    except Exception:
        sorted_filenames = sorted(list(grouped_results.keys()))

    for filename in sorted_filenames:
        if filename in grouped_results:
            # Sort the items for each file by model name to ensure consistent order
            items = sorted(grouped_results[filename], key=lambda x: x[1])
            final_table_data.extend(items)
            # Add a separator row for readability
            final_table_data.append(['---'] * len(headers))

    # Remove the last separator row
    if final_table_data:
        final_table_data.pop()

    print("\n--- Unified Side-by-Side Comparison ---")
    print(tabulate(final_table_data, headers=headers, tablefmt="grid", maxcolwidths=[40, 15, 15, 15, 80]))

else:
    print("❌ Error: Could not find 'native_results' or 'hf_results'.")
    print("Please make sure you have successfully run Cell #1 and Cell #2 before running this cell.")

--- Combining results from both models ---

--- Unified Side-by-Side Comparison ---
+------------------------------------------+----------------+-------------------+--------------+----------------------------------------------------------------------------------+
| Filename                                 | Model          | Avg. Confidence   | Perplexity   | Transcription                                                                    |
+==========================================+================+===================+==============+==================================================================================+
| Download (1)_audio.mp3                   | Hugging Face   | 0.0000            | 1.71         | It's an hour in and it's I'm pretty uncomfortable. You gotta be honest The first |
|                                          |                |                   |              | hour it was very difficult to try and Like what position do I want to just be in |
|               

In [ ]:
# @title Automated Analysis of ASR Model Results
# @markdown This script reads the output files from the previous step,
# @markdown compares the models based on the collected metrics, and
# @markdown prints a comprehensive summary of the findings.

import pandas as pd
import numpy as np

# --- 1. Configuration: File Paths ---
# These should match the output files from the previous script.
native_results_file = 'native_whisper_results.txt'
hf_results_file = 'huggingface_whisper_results.txt'

# --- 2. Data Loading and Parsing Function ---
def parse_results_file(filepath):
    """
    Parses the custom-formatted results text file into a pandas DataFrame.
    """
    try:
        # We skip the first 2 header lines and use '|' as the separator.
        df = pd.read_csv(
            filepath,
            sep='|',
            skiprows=2,
            header=None,
            names=['Filename', 'Confidence', 'Perplexity', 'Transcription'],
            # Remove whitespace around columns during parsing
            skipinitialspace=True
        )
        # Clean up the last column which may have trailing whitespace
        df['Transcription'] = df['Transcription'].str.strip()
        return df
    except FileNotFoundError:
        print(f"❌ Error: The file '{filepath}' was not found. Please ensure the previous script ran successfully.")
        return None
    except Exception as e:
        print(f"An error occurred parsing {filepath}: {e}")
        return None

# --- 3. Load the data into DataFrames ---
print("--- Loading and Parsing Result Files ---")
df_native = parse_results_file(native_results_file)
df_hf = parse_results_file(hf_results_file)

# --- 4. Data Cleaning and Processing ---
if df_native is not None and df_hf is not None:
    print("✅ Files loaded successfully. Cleaning and processing data...")
    # Merge the two dataframes on 'Filename' for direct comparison
    # We add suffixes to distinguish columns from each model
    df_comparison = pd.merge(
        df_native,
        df_hf,
        on='Filename',
        suffixes=('_native', '_hf')
    )

    # Convert columns to numeric types, forcing errors (like 'N/A' or 'ERROR') into NaN (Not a Number)
    for col in ['Confidence_native', 'Perplexity_native', 'Perplexity_hf']:
        df_comparison[col] = pd.to_numeric(df_comparison[col], errors='coerce')

    # --- 5. Perform Statistical Analysis ---
    print("\n--- STATISTICAL ANALYSIS ---")
    print("="*30)

    # Calculate average perplexity (ignoring any NaN values)
    avg_perplexity_native = df_comparison['Perplexity_native'].mean()
    avg_perplexity_hf = df_comparison['Perplexity_hf'].mean()

    print(f"\n📊 Average Perplexity (Lower is Better):")
    print(f"  - Native Whisper:   {avg_perplexity_native:.2f}")
    print(f"  - Hugging Face:     {avg_perplexity_hf:.2f}")

    if avg_perplexity_native < avg_perplexity_hf:
        print("  🏆 Winner: Native Whisper had a lower average perplexity.")
    elif avg_perplexity_hf < avg_perplexity_native:
        print("  🏆 Winner: Hugging Face had a lower average perplexity.")
    else:
        print("  ⚖️ It's a tie in average perplexity.")

    # Find the top 3 files with the WORST (highest) perplexity for each model
    print("\n\n🔥 Top 3 Files with Highest Perplexity (Potential Gibberish):")
    print("\n  --- Native Whisper ---")
    print(df_comparison.sort_values('Perplexity_native', ascending=False)[['Filename', 'Perplexity_native']].head(3).to_string(index=False))
    print("\n  --- Hugging Face Whisper ---")
    print(df_comparison.sort_values('Perplexity_hf', ascending=False)[['Filename', 'Perplexity_hf']].head(3).to_string(index=False))

    # Find the top 3 files with the LOWEST confidence score (only for native Whisper)
    print("\n\n🤔 Top 3 Files with Lowest Confidence (Native Whisper's Least Certain Transcriptions):")
    print(df_comparison.sort_values('Confidence_native', ascending=True)[['Filename', 'Confidence_native']].head(3).to_string(index=False))

    # --- 6. Detailed Side-by-Side Comparison ---
    print("\n\n--- DETAILED SIDE-BY-SIDE COMPARISON ---")
    print("="*40)
    # Display results for each file
    for index, row in df_comparison.iterrows():
        print(f"\n📂 FILENAME: {row['Filename']}")
        print("-"*(len(row['Filename'])+12))
        print(f"  Native Whisper:")
        print(f"    - Perplexity: {row['Perplexity_native']:.2f} | Confidence: {row['Confidence_native']:.4f}")
        print(f"    - Transcription: {row['Transcription_native']}")
        print(f"\n  Hugging Face:")
        print(f"    - Perplexity: {row['Perplexity_hf']:.2f}")
        print(f"    - Transcription: {row['Transcription_hf']}")
        print("-"*(len(row['Filename'])+12))


    # --- 7. Final Verdict ---
    print("\n\n--- FINAL VERDICT ---")
    print("="*20)
    print("Based on the automated analysis:")
    print("\n1.  **Analytical Capability:** The Native Whisper library is superior for analysis because it provides an easily accessible 'Confidence Score' out-of-the-box, allowing you to quickly identify problematic transcriptions without a ground truth. The Hugging Face pipeline does not offer this directly.")
    print("\n2.  **Linguistic Quality (Perplexity):**")
    if abs(avg_perplexity_native - avg_perplexity_hf) < 1: # A small tolerance for near-identical scores
        print("    The average perplexity scores were nearly identical, indicating both libraries produce text of very similar linguistic quality, as expected since they use the same underlying model.")
    elif avg_perplexity_native < avg_perplexity_hf:
        print(f"    The Native Whisper library produced text with a slightly lower average perplexity ({avg_perplexity_native:.2f} vs {avg_perplexity_hf:.2f}), suggesting a marginal advantage in generating natural-sounding text.")
    else:
        print(f"    The Hugging Face pipeline produced text with a slightly lower average perplexity ({avg_perplexity_hf:.2f} vs {avg_perplexity_native:.2f}), suggesting a marginal advantage in generating natural-sounding text.")
    print("\n**RECOMMENDATION:** For your goal of achieving crisp, clear text and having the ability to analyze transcription quality, the **Native `openai-whisper` library is the recommended choice.** Its transparency with confidence scores is a decisive feature for any in-depth project.")

else:
    print("\nAnalysis could not be completed due to file loading errors.")